Using this just for RandomForestClassifier - there's also ExtraTrees and RandomForestRegressors, but they didn't help my OOB, so they're not getting run here...

In [1]:
import os
import sys
import operator
import numpy as np
import pandas as pd
from scipy import sparse
import xgboost as xgb
import random
from sklearn import model_selection, preprocessing, ensemble
from sklearn.metrics import log_loss
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

import pickle
import time

import sklearn.cluster

import Levenshtein

from multiprocessing import Pool

import lightgbm as lgbm

In [2]:
train_df = pd.read_pickle('fin-dprep-train.pkl')
test_df = pd.read_pickle('fin-dprep-test.pkl')
features_to_use = pickle.load(open('fin-dprep-flist.pkl', 'rb'))

In [3]:
medium_price = pd.read_pickle('fin-medium-price.pkl')

train_df = pd.merge(train_df, medium_price, left_on='listing_id', right_index=True)
test_df = pd.merge(test_df, medium_price, left_on='listing_id', right_index=True)



In [4]:
for df in [train_df, test_df]:
    df['predicted_price_diff'] = np.log(df.predicted_price) - np.log(df.price)
    df['predicted_price_ratio'] = np.log(df.predicted_price) / np.log(df.price)

In [5]:
# fill in the NaN's.

for t in train_df.keys():
    nacount = train_df[t].isnull().sum()
    if nacount:
#        nacount_test = test_df[t].isnull().sum()
        print(t, nacount / len(train_df))#, nacount_test / len(test_df))
        
train_df.fillna(0, inplace=True)
test_df.fillna(0, inplace=True)


price_group 0.0488733992543
price_ratio 0.0488733992543
manager_shortdesc_rate 0.0688725887502
manager_building0_rate 0.0688725887502
manager_0feature_rate 0.0688725887502
manager_median_price 0.0688725887502
manager_lazy_rate 0.0688725887502


In [6]:
class MeansProcessor:
    def __init__(self, key, outkey = None, tgt = 'interest'):
        self.key = key
        self.outkey = key if outkey is None else outkey
        
        self.count = {}
        self.means = {}
        self.std = {}
        self.global_means = {}
        
        self.tgt = tgt
        
        self.outkeys = [self.outkey + '_level', self.outkey + '_level_std']
        
    def fit(self, df):
        self.global_means[self.outkey + '_level'] = df[self.tgt].mean()
        self.global_means[self.outkey + '_level_std'] = df[self.tgt].std()
            
        for k in df.groupby(self.key, sort=False):
            
            self.count[k[0]] = len(k[1])

            if len(k[1]) < 0:
                self.means[k[0]] = np.nan
                self.std[k[0]] = np.nan
            else:
                self.means[k[0]] = np.mean(k[1][self.tgt])
                self.std[k[0]] = np.std(k[1][self.tgt])
            
    def predict(self, df, nans = False):
        for l in self.outkeys:
            df[l] = np.nan if nans else self.global_means[l]
            
        df[self.outkey + '_count'] = 0
            
        for k in df.groupby(self.key, sort=False):
            if k[0] == 0:
                continue
            
            if k[0] in self.means:
                df.loc[k[1].index, self.outkey + '_count'] = self.count[k[0]]
                df.loc[k[1].index, self.outkey + '_level'] = self.means[k[0]]
                df.loc[k[1].index, self.outkey + '_level_std'] = self.std[k[0]]
        
        return df
    
    def get_features(self):
        return self.outkeys.copy() + [self.outkey + '_count']

# i kept the same index randomization (with fixed seed) so I could validate this code against
# the original...

target_num_map = {'low':0, 'medium':1, 'high':2}
train_y = np.array(train_df['interest_level'].apply(lambda x: target_num_map[x]))

def proc_fold(fold):
    train_index = fold[0]
    test_index = fold[1]
    
    cv_train = train_df.iloc[train_index]
    cv_valid = train_df.iloc[test_index][['interest_level', 'manager_id', 'building_id']]
    cv_test = test_df.copy()
    
    m_build = MeansProcessor('building_id', 'building_sort')
    m_build.fit(cv_train)
    cv_valid = m_build.predict(cv_valid)
    cv_test = m_build.predict(cv_test)

    m_mgr = MeansProcessor('manager_id', 'manager_sort')
    m_mgr.fit(cv_train)
    cv_valid = m_mgr.predict(cv_valid)
    cv_test = m_mgr.predict(cv_test)

    m_comb = MeansProcessor(['building_id', 'manager_id'], 'mb_comb')
    m_comb.fit(cv_train)
    cv_valid = m_comb.predict(cv_valid)
    cv_test = m_comb.predict(cv_test)

    return cv_train, cv_valid, cv_test

kf = model_selection.StratifiedKFold(n_splits=5, shuffle=True, random_state=2016)
folds = [(k[0], k[1]) for k in kf.split(list(range(train_df.shape[0])), train_y)]

#with Pool(5) as pool:
#    rv = pool.map(proc_fold, folds)

import pickle

try:
    rv = pickle.load(open('0422-model-groupfeatures_nonan.pkl', 'rb'))
except:
    with Pool(5) as pool:
        rv = pool.map(proc_fold, folds)

        pickle.dump(rv, open('0422-model-groupfeatures_nonan.pkl', 'wb'))

# dummies to get feature id's
m_build = MeansProcessor('building_id', 'building_sort')
m_mgr = MeansProcessor('manager_id', 'manager_sort')
m_comb = MeansProcessor(['building_id', 'manager_id'], 'mb_comb')

group_features = m_build.get_features() + m_mgr.get_features() + m_comb.get_features()

#cv_test = [r[2] for r in rv]
cv_test = []
for r in rv:
    cv_test.append(test_df.merge(r[2][group_features], left_index=True, right_index=True))

cv_allvalid = pd.concat([r[1] for r in rv])

train_df = train_df.merge(cv_allvalid[group_features], left_index=True, right_index=True)

In [7]:
kf = model_selection.StratifiedKFold(n_splits=5, shuffle=True, random_state=2016)
folds = [(k[0], k[1]) for k in kf.split(list(range(train_df.shape[0])), train_df.interest_cat)]


In [8]:
for df in [train_df] + cv_test:
    df['price_t'] = df['price_t'].clip(0, 13000)
    df['price_per_room'] = df['price_per_room'].clip(0, 13000)
    #df['density_lin005'] = df['density_lin005'].clip(-50, 50)
    df['predicted_price_ratio'] = df['predicted_price_ratio'].clip(-50, 50)

In [9]:
train_df_normalized = train_df.copy()
cvtest_normalized = [df.copy() for df in cv_test]

train_df_normalized['listing_id_norm'] = train_df_normalized['listing_id']
for df in cvtest_normalized:
    df['listing_id_norm'] = df['listing_id']

normalized_keys = []

scaler = {}
for f in train_df.keys():
    if f[0:2] == 'f_' or f[0:3] == 'fm_':
        train_df_normalized[f] = train_df_normalized[f].clip(0, 1)
        for df in cvtest_normalized:
            df[f] = df[f].clip(0, 1)
    elif 'interest' in f or f == 'listing_id' or f == 'index':
        continue
    elif f == 'created' or train_df[f].dtype == 'O':
        train_df_normalized.drop(f, axis=1, inplace=True)
        for df in cvtest_normalized:
            df.drop(f, axis=1, inplace=True)
        continue
    else:
        #print(f, train_df[f].min(), train_df[f].max(), test_df[f].min(), test_df[f].max())
        scaler[f] = sklearn.preprocessing.StandardScaler()
        train_df_normalized[f] = scaler[f].fit_transform(train_df_normalized[f].values.reshape(-1,1))[:,0]
        for df in cvtest_normalized:
            df[f] = scaler[f].transform(df[f].values.reshape(-1,1))[:,0]
        
    normalized_keys.append(f)
        

/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:431: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:431: DataConversionWarning: Data with input dtype uint8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


models begin here

In [10]:
fl = normalized_keys.copy() + m_build.get_features() + m_mgr.get_features() 

#for f in ['density_exp01', 'density_exp005', 'density_lin005', 'density_gaussian001', 'density_gaussian', 'density_gaussian01', 'density_gaussian02', 'density_gaussian04']:
#    fl.remove(f)
    
#fl.append('density_gaussian02')
#fl.append('density_exp01')


fl.remove('predicted_price_ratio')
fl.remove('manager_building0_rate')
fl.remove('manager_shortdesc_rate')
fl.remove('manager_0feature_rate')
#fl.append('manager_sort_count')


In [11]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.datasets import load_digits
from sklearn.ensemble import RandomForestClassifier

from scipy.stats import randint as sp_randint


In [12]:
# RandomForestClassifier

start = time.time()

# prep CV

cv_train = []
cv_valid = []

for tr_index, val_index in kf.split(train_df.index, train_df.interest_cat):
        cv_train.append(train_df_normalized.loc[tr_index])
        cv_valid.append(train_df_normalized.loc[val_index])

models = []
cv_preds = []
df_cvpreds = []

for f in range(5):
    #dset = lgbm.Dataset(cv_train[f][fl], cv_train[f].interest_cat, silent=True)
    #dset_val = lgbm.Dataset(cv_valid[f][fl], cv_valid[f].interest_cat, silent=True)
    #models.append(lgbm.train(lgbm_params, dset, early_stopping_rounds=100, verbose_eval=False, valid_sets=dset_val, num_boost_round=2000))

    models.append(sklearn.ensemble.RandomForestClassifier(n_estimators=10, min_samples_split=8, min_samples_leaf=4, 
                                                          n_jobs=-1, class_weight=None, random_state=0))
    models[-1].fit(cv_train[f][fl].values, cv_train[f].interest_cat.values)
    
    best = None
    
    for nest in range(20, 2000, 10):
        models[-1].set_params(warm_start = True, n_estimators=nest)
        models[-1].fit(cv_train[f][fl].values, cv_train[f].interest_cat.values)
    
        preds = models[-1].predict_proba(cv_valid[f][fl].values)
        score = sklearn.metrics.log_loss(cv_valid[f].interest_cat, preds)
        
        print(nest, score)

        if best is None or score < best[0]:
            best = (score, nest)
        elif nest - best[1] >= 50:
            break
            
    #models[-1].set_params(n_estimators = best[1])
    
    #print('done training')
    
    cv_preds.append(models[-1].predict_proba(cv_valid[f][fl].values))

    df_cvpreds.append(pd.DataFrame(cv_preds[f], columns=['low', 'medium', 'high']))
    df_cvpreds[f].index = cv_valid[f].index
    df_cvpreds[f]['interest_cat'] = cv_valid[f].interest_cat
    df_cvpreds[f]['listing_id'] = cv_valid[f].listing_id

    df_cvpreds[f].set_index('listing_id', inplace=True)
    print(f, sklearn.metrics.log_loss(df_cvpreds[f].interest_cat, df_cvpreds[f][['low', 'medium', 'high']]))

df_cvpreds = pd.concat(df_cvpreds)

tgts = ['low', 'medium', 'high']

print('combined: ', sklearn.metrics.log_loss(df_cvpreds.interest_cat, df_cvpreds[tgts]))

end = time.time()
print(end  - start)

20 0.583326595675
30 0.564583867318
40 0.55640282163
50 0.55297015512
60 0.5522085791
70 0.551036046135
80 0.548456980448
90 0.548120494467
100 0.547398887508
110 0.544103625174
120 0.543885010018
130 0.544056941458
140 0.54390063626
150 0.543760753053
160 0.543396538985
170 0.543209339341
180 0.543164062785
190 0.542978549216
200 0.542855772845
210 0.543000699285
220 0.542942881345
230 0.542866927348
240 0.542786932211
250 0.542523391965
260 0.542565326172
270 0.542638907563
280 0.54240001092
290 0.542447548681
300 0.542322008059
310 0.542182701203
320 0.542270802642
330 0.542336732247
340 0.542359290234
350 0.542300408339
360 0.54224689899
0 0.54225151966
20 0.562783574225
30 0.548940895073
40 0.54181185985
50 0.541158801244
60 0.536130767212
70 0.535276797363
80 0.534634191309
90 0.533877558573
100 0.533935561793
110 0.533680906437
120 0.533434391824
130 0.533563194397
140 0.533453484687
150 0.533840530709
160 0.533867656229
170 0.534021518884
1 0.534020801947
20 0.581065021749
30 0

In [15]:
testpreds = []
for i, m in enumerate(models):
    testpreds.append(m.predict_proba(cvtest_normalized[i][fl]))
    
f = np.array(testpreds).mean(axis=0)
df_testpreds = pd.DataFrame(f, columns=['low', 'medium', 'high'])
df_testpreds.index = test_df.index
df_testpreds['listing_id'] = test_df.listing_id
df_testpreds.set_index('listing_id', inplace=True)

df_output = pd.concat([df_testpreds, df_cvpreds[tgts]])
df_output.sort_index(inplace=True)


In [16]:
df_fold = []
for i in range(len(testpreds)):
    df_fold.append(pd.DataFrame(testpreds[i]))
    df_fold[-1]['listing_id'] = test_df.listing_id
    df_fold[-1].sort_values('listing_id', inplace=True)
    df_fold[-1].set_index('listing_id', inplace=True)

pickle.dump((df_output, df_fold), open('model-output-rf.pkl', 'wb'))

In [17]:
# RandomForestRegressor

start = time.time()

# prep CV

cv_train = []
cv_valid = []

for tr_index, val_index in kf.split(train_df.index, train_df.interest_cat):
        cv_train.append(train_df_normalized.loc[tr_index])
        cv_valid.append(train_df_normalized.loc[val_index])

models = []
cv_preds = []
df_cvpreds = []

for f in range(5):
    #dset = lgbm.Dataset(cv_train[f][fl], cv_train[f].interest_cat, silent=True)
    #dset_val = lgbm.Dataset(cv_valid[f][fl], cv_valid[f].interest_cat, silent=True)
    #models.append(lgbm.train(lgbm_params, dset, early_stopping_rounds=100, verbose_eval=False, valid_sets=dset_val, num_boost_round=2000))

    models.append(sklearn.ensemble.RandomForestRegressor(n_estimators=10, min_samples_split=8, min_samples_leaf=4, 
                                                          n_jobs=-1, random_state=0))
    models[-1].fit(cv_train[f][fl].values, cv_train[f].interest_cat.values)
    
    best = None
    
    for nest in range(20, 2000, 10):
        models[-1].set_params(warm_start = True, n_estimators=nest)
        models[-1].fit(cv_train[f][fl].values, cv_train[f].interest.values)
    
        preds = models[-1].predict(cv_valid[f][fl].values)
        #score = sklearn.metrics.log_loss(cv_valid[f].interest, preds)
        score = np.sqrt(sklearn.metrics.mean_squared_error(cv_valid[f].interest, preds))
        
        print(nest, score)

        if best is None or score < best[0]:
            best = (score, nest)
        elif nest - best[1] >= 50:
            break
            
    #models[-1].set_params(n_estimators = best[1])
    
    #print('done training')
    
    cv_preds.append(models[-1].predict(cv_valid[f][fl].values))

    df_cvpreds.append(pd.DataFrame(cv_preds[f], columns=['prediction']))
    df_cvpreds[f].index = cv_valid[f].index
    df_cvpreds[f]['interest'] = cv_valid[f].interest
    df_cvpreds[f]['listing_id'] = cv_valid[f].listing_id

    df_cvpreds[f].set_index('listing_id', inplace=True)
    print(f) #, np.sqrt(sklearn.metrics.mean_squared_error(cv_valid[f].interest, preds)))

df_cvpreds = pd.concat(df_cvpreds)

tgts = ['low', 'medium', 'high']

print('combined: ', np.sqrt(sklearn.metrics.mean_squared_error(df_cvpreds.interest, df_cvpreds.prediction))) #sklearn.metrics.log_loss(df_cvpreds.interest_cat, df_cvpreds[tgts]))

end = time.time()
print(end  - start)

20 0.292096527436
30 0.27158341902
40 0.264031400668
50 0.260278668073
60 0.258340367339
70 0.257256241264
80 0.256278995325
90 0.255620284908
100 0.255145913323
110 0.254906806619
120 0.254613388379
130 0.254556479169
140 0.254476706799
150 0.254357550832
160 0.254319593032
170 0.254194507073
180 0.254108090616
190 0.253921341687
200 0.253864447126
210 0.253780455416
220 0.253819769413
230 0.253767379979
240 0.253785431681
250 0.253761105406
260 0.253740080722
270 0.253761059263
280 0.25375415227
290 0.253710537549
300 0.253675982788
310 0.253636129461
320 0.253587363658
330 0.253578664685
340 0.253577816128
350 0.253545655204
360 0.253587681629
370 0.253598081195
380 0.253641764732
390 0.253668734815
400 0.253672682618
0
20 0.288663671028
30 0.269145129711
40 0.261705426042
50 0.257644651819
60 0.255716143771
70 0.254316551319
80 0.253379513614
90 0.252792486215
100 0.252399287617
110 0.252110819435
120 0.25183553323
130 0.251595262932
140 0.2514430371
150 0.251363101497
160 0.251331

In [18]:
testpreds = []
for i, m in enumerate(models):
    testpreds.append(m.predict(cvtest_normalized[i][fl]))
    

In [19]:
f = np.array(testpreds).mean(axis=0)
df_testpreds = pd.DataFrame(f, columns=['prediction'])
df_testpreds.index = test_df.index
df_testpreds['listing_id'] = test_df.listing_id
df_testpreds.set_index('listing_id', inplace=True)

df_output = pd.concat([df_testpreds, df_cvpreds[['prediction']]])
df_output.sort_index(inplace=True)

df_output.to_pickle('bag-model-rfr-v1.pkl')

df_fold = []
for i in range(len(testpreds)):
    df_fold.append(pd.DataFrame(testpreds[i]))
    df_fold[-1]['listing_id'] = test_df.listing_id
    df_fold[-1].sort_values('listing_id', inplace=True)
    df_fold[-1].set_index('listing_id', inplace=True)

pickle.dump((df_output, df_fold), open('bagta-model-rfr-v1.pkl', 'wb'))

In [23]:
# ExtraTreesRegressor 

start = time.time()

# prep CV

cv_train = []
cv_valid = []

for tr_index, val_index in kf.split(train_df.index, train_df.interest_cat):
        cv_train.append(train_df_normalized.loc[tr_index])
        cv_valid.append(train_df_normalized.loc[val_index])

models = []
cv_preds = []
df_cvpreds = []

for f in range(5):
    models.append(sklearn.ensemble.ExtraTreesRegressor(n_estimators=10, min_samples_split=8, min_samples_leaf=4, 
                                                          n_jobs=-1, random_state=0))
    models[-1].fit(cv_train[f][fl].values, cv_train[f].interest_cat.values)
    
    best = None
    
    for nest in range(20, 2000, 10):
        models[-1].set_params(warm_start = True, n_estimators=nest)
        models[-1].fit(cv_train[f][fl].values, cv_train[f].interest.values)
    
        preds = models[-1].predict(cv_valid[f][fl].values)
        #score = sklearn.metrics.log_loss(cv_valid[f].interest, preds)
        score = np.sqrt(sklearn.metrics.mean_squared_error(cv_valid[f].interest, preds))
        
        print(nest, score)

        if best is None or score < best[0]:
            best = (score, nest)
        elif nest - best[1] >= 50:
            break
            
    cv_preds.append(models[-1].predict(cv_valid[f][fl].values))

    df_cvpreds.append(pd.DataFrame(cv_preds[f], columns=['prediction']))
    df_cvpreds[f].index = cv_valid[f].index
    df_cvpreds[f]['interest'] = cv_valid[f].interest
    df_cvpreds[f]['listing_id'] = cv_valid[f].listing_id

    df_cvpreds[f].set_index('listing_id', inplace=True)
    print(f) #, np.sqrt(sklearn.metrics.mean_squared_error(cv_valid[f].interest, preds)))

df_cvpreds = pd.concat(df_cvpreds)

tgts = ['low', 'medium', 'high']

print('combined: ', np.sqrt(sklearn.metrics.mean_squared_error(df_cvpreds.interest, df_cvpreds.prediction))) #sklearn.metrics.log_loss(df_cvpreds.interest_cat, df_cvpreds[tgts]))

end = time.time()
print(end  - start)

20 0.284186195488
30 0.265958821954
40 0.259247013231
50 0.255700746787
60 0.254181923445
70 0.253214218752
80 0.252695012866
90 0.252027572135
100 0.251631224067
110 0.251455173568
120 0.251306111599
130 0.251134836482
140 0.251014448712
150 0.250942882723
160 0.250903977167
170 0.250743492843
180 0.250684503264
190 0.250643587619
200 0.250566978491
210 0.25058266343
220 0.25055057091
230 0.250530526918
240 0.250527243495
250 0.250551237175
260 0.250560912678
270 0.250485336496
280 0.250482299271
290 0.250537845928
300 0.250539261569
310 0.250481284099
320 0.250469261469
330 0.250471813276
340 0.250442490443
350 0.250460725332
360 0.250453082106
370 0.250426486544
380 0.250404520127
390 0.250404315517
400 0.250401116492
410 0.250393721715
420 0.250360003323
430 0.250321644975
440 0.250371296554
450 0.250378230768
460 0.250378768944
470 0.250390197725
480 0.250379463062
0
20 0.277909206138
30 0.260961212184
40 0.254957303532
50 0.252181429124
60 0.25054498715
70 0.249493731192
80 0.248

In [24]:
testpreds = []
for i, m in enumerate(models):
    testpreds.append(m.predict(cvtest_normalized[i][fl]))
    
f = np.array(testpreds).mean(axis=0)
df_testpreds = pd.DataFrame(f, columns=['prediction'])
df_testpreds.index = test_df.index
df_testpreds['listing_id'] = test_df.listing_id
df_testpreds.set_index('listing_id', inplace=True)

df_output = pd.concat([df_testpreds, df_cvpreds[['prediction']]])
df_output.sort_index(inplace=True)

#df_output.to_pickle('0423-model-etr-v1.pkl')

df_fold = []
for i in range(len(testpreds)):
    df_fold.append(pd.DataFrame(testpreds[i]))
    df_fold[-1]['listing_id'] = test_df.listing_id
    df_fold[-1].sort_values('listing_id', inplace=True)
    df_fold[-1].set_index('listing_id', inplace=True)

pickle.dump((df_output, df_fold), open('bagta-model-etr-v1.pkl', 'wb'))

In [103]:
df_output

,prediction
listing_id,
6811957,0.410999
6811958,0.609036
6811960,0.409615
6811964,0.538739
6811965,0.683138
6811966,0.615311
6811971,0.345358
6811973,0.376863
6811974,0.515847


In [79]:
fl = ['price', 'manager_sort_level', 'latitude', 'longitude', 'bedrooms', 'bathrooms', 'density_exp005', 'predicted_price_diff', 'created_hour']


# ET

# RandomForestClassifier

start = time.time()

# prep CV

cv_train = []
cv_valid = []

for tr_index, val_index in kf.split(train_df.index, train_df.interest_cat):
        cv_train.append(train_df_normalized.loc[tr_index])
        cv_valid.append(train_df_normalized.loc[val_index])

models = []
cv_preds = []
df_cvpreds = []

for f in range(5):
    #dset = lgbm.Dataset(cv_train[f][fl], cv_train[f].interest_cat, silent=True)
    #dset_val = lgbm.Dataset(cv_valid[f][fl], cv_valid[f].interest_cat, silent=True)
    #models.append(lgbm.train(lgbm_params, dset, early_stopping_rounds=100, verbose_eval=False, valid_sets=dset_val, num_boost_round=2000))

    models.append(sklearn.ensemble.ExtraTreesClassifier(n_estimators=10, max_features=len(fl),
                                                          n_jobs=-1, class_weight=None, random_state=0))
    models[-1].fit(cv_train[f][fl].values, cv_train[f].interest_cat.values)
    
    best = None
    
    for nest in range(20, 2000, 10):
        models[-1].set_params(warm_start = True, n_estimators=nest)
        models[-1].fit(cv_train[f][fl].values, cv_train[f].interest_cat.values)
    
        preds = models[-1].predict_proba(cv_valid[f][fl].values)
        score = sklearn.metrics.log_loss(cv_valid[f].interest_cat, preds)
        
        print(nest, score)

        if best is None or score < best[0]:
            best = (score, nest)
        elif nest - best[1] >= 50:
            break
            
    #models[-1].set_params(n_estimators = best[1])
    
    #print('done training')
    
    cv_preds.append(models[-1].predict_proba(cv_valid[f][fl].values))

    df_cvpreds.append(pd.DataFrame(cv_preds[f], columns=['low', 'medium', 'high']))
    df_cvpreds[f].index = cv_valid[f].index
    df_cvpreds[f]['interest_cat'] = cv_valid[f].interest_cat
    df_cvpreds[f]['listing_id'] = cv_valid[f].listing_id

    df_cvpreds[f].set_index('listing_id', inplace=True)
    print(f, sklearn.metrics.log_loss(df_cvpreds[f].interest_cat, df_cvpreds[f][['low', 'medium', 'high']]))

df_cvpreds = pd.concat(df_cvpreds)

tgts = ['low', 'medium', 'high']

print('combined: ', sklearn.metrics.log_loss(df_cvpreds.interest_cat, df_cvpreds[tgts]))

end = time.time()
print(end  - start)

20 1.12296132847
30 0.942927949061
40 0.844550481435
50 0.797762267704
60 0.781118544176
70 0.74291014053
80 0.735813953207
90 0.725778708524
100 0.70352403112
110 0.691069629282
120 0.678172029392
130 0.67497619837
140 0.674740438006
150 0.671048130923
160 0.6705096715
170 0.670215898684
180 0.669729628895
190 0.666112938059
200 0.663259300476
210 0.659661933232
220 0.650185280136
230 0.649839284451
240 0.643923314802
250 0.641127095932
260 0.641433678261
270 0.641525789111
280 0.641486492967
290 0.638562517295
300 0.638519512836
310 0.635716730599
320 0.632999902856
330 0.629937480721
340 0.626842628871
350 0.623850249736
360 0.62094618567
370 0.620845163869
380 0.621167801268
390 0.621128292064
400 0.621091840209
410 0.620958795269
420 0.620816925182
430 0.6179475613
440 0.617827747385
450 0.617820263081
460 0.617769334832
470 0.6176277653
480 0.617659207137
490 0.617575733492
500 0.617419231484
510 0.617287101022
520 0.617173388505
530 0.617194686212
540 0.61432017067
550 0.6115886

KeyboardInterrupt: 

In [34]:
train_df['desc_xp_count'] = train_df.description.apply(lambda x: x.count('!'))
train_df['desc_xp_ratio'] = train_df.description.apply(lambda x: (x.count('!') / len(x)) if len(x) else 0)
train_df['desc_xp_first'] = train_df.description.apply(lambda x: x.find('!') if len(x) else 0)
train_df['desc_xp_inv_first'] = train_df.description.apply(lambda x: (len(x) - x.find('!')) if len(x) else 0)
train_df['desc_xp_inv_mult'] = train_df.desc_xp_count * train_df.desc_xp_inv_first

In [30]:
# prep CV

cv_train = []
cv_valid = []

for tr_index, val_index in kf.split(train_df.index, train_df.interest_cat):
        cv_train.append(train_df.loc[tr_index])
        cv_valid.append(train_df.loc[val_index])

In [74]:
t4_params = {
    'boosting_type': 'gbdt', 'objective': 'multiclass', 'nthread': -1, 'silent': True,
    'num_leaves': 2**5, 'learning_rate': 0.02, 'max_depth': -1, 'metric': ['multi_logloss'],
    'max_bin': 255, 'subsample_for_bin': 50000,
    'subsample': 0.8, 'subsample_freq': 1, 'colsample_bytree': 0.6, 'reg_alpha': 1, 'reg_lambda': 0,
    'min_split_gain': 0.25, 'min_child_weight': .5, 'min_child_samples': 20, 'scale_pos_weight': 1}

lgbm_params = t4_params.copy()
lgbm_params['num_class'] = 3


In [78]:
fl = ['price', 'manager_sort_level', 'latitude', 'longitude', 'bedrooms', 'bathrooms', 'density_exp005', 'predicted_price_diff', 'created_hour']

start = time.time()

# prep CV

cv_train = []
cv_valid = []

for tr_index, val_index in kf.split(train_df.index, train_df.interest_cat):
        cv_train.append(train_df_normalized.loc[tr_index])
        cv_valid.append(train_df_normalized.loc[val_index])

models = []
cv_preds = []
df_cvpreds = []

for f in range(5):
    dset = lgbm.Dataset(cv_train[f][fl], cv_train[f].interest_cat, silent=True)
    dset_val = lgbm.Dataset(cv_valid[f][fl], cv_valid[f].interest_cat, silent=True)
    models.append(lgbm.train(lgbm_params, dset, early_stopping_rounds=100, verbose_eval=False, valid_sets=dset_val, num_boost_round=2000))

    #print('done training')
    
    cv_preds.append(models[-1].predict(cv_valid[f][fl]))

    df_cvpreds.append(pd.DataFrame(cv_preds[f], columns=['low', 'medium', 'high']))
    df_cvpreds[f].index = cv_valid[f].index
    df_cvpreds[f]['interest_cat'] = cv_valid[f].interest_cat
    df_cvpreds[f]['listing_id'] = cv_valid[f].listing_id

    df_cvpreds[f].set_index('listing_id', inplace=True)
    print(f, sklearn.metrics.log_loss(df_cvpreds[f].interest_cat, df_cvpreds[f][['low', 'medium', 'high']]))

df_cvpreds = pd.concat(df_cvpreds)

tgts = ['low', 'medium', 'high']

print('combined: ', sklearn.metrics.log_loss(df_cvpreds.interest_cat, df_cvpreds[tgts]))

end = time.time()
print(end  - start)

0 0.552516080553
1 0.557726118266
2 0.561741087415
3 0.550305671681
4 0.556885845
combined:  0.555834896702
30.58432674407959


In [77]:
testpreds = []
for i, m in enumerate(models):
    testpreds.append(m.predict(cv_test[i][fl], num_iteration=m.best_iteration))
    


f = np.array(testpreds).mean(axis=0)
df_testpreds = pd.DataFrame(f, columns=['low', 'medium', 'high'])
df_testpreds.index = test_df.index
df_testpreds['listing_id'] = test_df.listing_id
df_testpreds.set_index('listing_id', inplace=True)

df_output = pd.concat([df_testpreds, df_cvpreds[tgts]])
df_output.sort_index(inplace=True)

df_output.to_pickle('0423-minimod-lgbm-lf-v1.pkl')

In [23]:
corrs = []
for k in fl:
    corrs.append((k, train_df[k].corr(train_df.interest)))

In [24]:
sorted(corrs, key=operator.itemgetter(1))

[('time_stamp', -0.18019039565076073),
 ('display_address', -0.10638217137972825),
 ('address_ratio', -0.08435685614698217),
 ('half_bathroom', -0.082952509296588234),
 ('bathrooms', -0.08290129995713677),
 ('f_doorman', -0.080408870770706103),
 ('f_dogs_allowed', -0.069394270715112102),
 ('f_cats_allowed', -0.063240578842069212),
 ('f_garage', -0.061826833663457474),
 ('f_fitness_center', -0.041625294080062145),
 ('created_day', -0.035318942170163382),
 ('f_hardwood', -0.034737405598447645),
 ('f_fireplace', -0.028046029385565645),
 ('listing_id', -0.027602467182586288),
 ('density', -0.025715142282351982),
 ('price', -0.024422071937294336),
 ('f_gym_fitness', -0.022982979101316876),
 ('f_childrens_playroom', -0.022148888512394029),
 ('created_dayofyear', -0.01735538096319977),
 ('f_garden', -0.015741001576469776),
 ('f_bike_room', -0.01528765527419634),
 ('f_indoor_pool', -0.015284345235786326),
 ('f_brownstone', -0.014610002072087057),
 ('f_in_unit_washer_dryer', -0.0143146749879894

In [25]:
pd.Series.corr?

In [38]:
train_df[~(train_df.f_doorman == 1)].price.mean()

3184.9364928241475